---
comments: true
layout: post
title: ML Project - Individual Review Ticket
description: Review Ticket for ML custom dataset.
type: tangibles
courses: { compsci: {week: 28} }
---

### Introduction

The topics for each of our (Aditya, Varun, Tucker) CPT projects collectively came to essentially stocks/business and cooking/food. So, we found a suitable dataset of ordered items from a bakery to combine those elements of business and food/cooking/baking, deciding that our ML model would determine a food order at the bakery (ex: "Bread", "Coffee", "Pastry", etc.) based on the other variables in the dataset (exact military time, general time: "Afternoon/Morning", and day of the week: "Weekend/Weekday").

Because our dataset of ~20 thousand total rows/orders included nearly 1/4 "Coffee" as a food item, it was extremely common to get Coffee"" rather than any other item. Therefore, we made two separate Models, APIs, and frontend input fields to include both the normal (skewed by "Coffee") dataset/CSV file, and another identitical input field that simply used a different Model, API, and dataset and different datset/CSV file with "Coffee" filtered out. Screenshots below.

### Project Screenshots

<img src="ML1">

<img src="ML2">

<img src="ML3">

<img src="ML4">